# 🌐 P2P Live Simulation

Use `P2PStub` to spin up several clients and the aggregator in-process. Track messages sent/received per client over time.

In [ ]:
import time
import threading
        import matplotlib.pyplot as plt
from polyscale_fl.networking.p2p_stub import P2PStub


In [ ]:
# Setup P2P bus
bus = P2PStub()
        NUM_CLIENTS = 5
clients = [f"client_{i}" for i in range(NUM_CLIENTS)]
        message_counts = {cid: 0 for cid in clients}
        lock = threading.Lock()
        def handler_factory(cid):
    def handler(from_id, payload):
        with lock:
            message_counts[cid] += 1
    return handler
        # register handlers
for cid in clients:
    bus.register(cid, handler_factory(cid))
        # designate aggregator
bus.register("aggregator", lambda f, p: None)
bus.set_aggregator("aggregator")

In [ ]:
# Sender thread: each client periodically sends a message to random other peer
import random
def sender(cid):
    while True:
        peers = bus.get_peers()
        peers = [p for p in peers if p != cid]
        if not peers:
            time.sleep(0.1)
            continue
        target = random.choice(peers)
        bus.send(cid, target, {"type": "ping", "from": cid})
        time.sleep(random.uniform(0.05, 0.2))
        threads = []
for cid in clients:
    t = threading.Thread(target=sender, args=(cid,), daemon=True)
    t.start()
    threads.append(t)


## Run simulation for some seconds, plot message counts

In [ ]:
RUN_SEC = 5
time.sleep(RUN_SEC)
        # Plot counts
plt.figure(figsize=(8,4))
plt.bar(message_counts.keys(), message_counts.values())
plt.xticks(rotation=45)
plt.ylabel('Messages received')
plt.title(f'P2P message counts after {RUN_SEC} seconds')
plt.show()

## Observation
You can tweak number of clients, send frequency, or introduce artificial delays in `P2PStub` to study effect on topology or load.